In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
import requests
import time
import json

In [3]:
# you need to install Biopython:
# pip install biopython

# Full discussion:
# https://marcobonzanini.wordpress.com/2015/01/12/searching-pubmed-with-python/

from Bio import Entrez

term = "fever" # topic of interest (string)
retmax = '10'
mindate = 2012 # beginning date search parameter. takes form YYYY/MM/DD, YYYY/MM, or YYYY
maxdate = 2012 # ending date search parameter. takes form YYYY/MM/DD, YYYY/MM, or YYYY
#reldate = False # searches the last N days (int)

def search(query, retmax=False, mindate=False, maxdate=False, reldate=False): 
    Entrez.email = 'lukeam2929@gmail.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax=retmax,
                            retmode='xml',
                            datetype='pdat',
                            #reldate=reldate,
                            mindate=mindate,
                            maxdate=maxdate,
                            term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'your.email@example.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

if __name__ == '__main__':
    results = search(term, retmax, mindate, maxdate) # add reldate if needed
    id_list = results['IdList']
    papers = fetch_details(id_list)
#    for i, paper in enumerate(papers):
#        print("%d) %s" % (i+1, paper['MedlineCitation']['Article']['ArticleTitle']))

1) [Fever and cancer: components of diagnosis for optimal management].
2) Urinary tract infections and post-operative fever in percutaneous nephrolithotomy.
3) Fever effects and treatment in critical care: literature review.
4) Different fever definitions and the rate of fever and neutropenia diagnosed in children with cancer: a retrospective two-center cohort study.
5) Is immediate postoperative fever related to drop in haemoglobin? A comparative study in simultaneous bilateral total knee arthroplasty patients.
6) A review of the physiology of fever in birds.
7) An adolescent boy with acute kidney injury and fever. Hemorrhagic fever with renal syndrome (HFRS).
8) [Fever as the presenting manifestation of colon cancer: a case series of 11 patients].
9) Effect of fever on hospital presentation, diagnosis, and treatment in patients with H1N1/09 influenza.
10) Fever, irritability, and ulcerations in an infant.


In [4]:
# print first paper in full
# print(json.dumps(papers[0], indent=2, separators=(',', ':')))

{
  "MedlineCitation":{
    "DateCompleted":{
      "Month":"03",
      "Day":"31",
      "Year":"2014"
    },
    "OtherID":[],
    "MeshHeadingList":[
      {
        "QualifierName":[
          "physiology"
        ],
        "DescriptorName":"Body Temperature Regulation"
      },
      {
        "QualifierName":[
          "complications"
        ],
        "DescriptorName":"Cross Infection"
      },
      {
        "QualifierName":[],
        "DescriptorName":"Diagnosis, Differential"
      },
      {
        "QualifierName":[
          "complications",
          "diagnosis",
          "therapy"
        ],
        "DescriptorName":"Fever"
      },
      {
        "QualifierName":[],
        "DescriptorName":"Humans"
      },
      {
        "QualifierName":[
          "complications",
          "diagnosis",
          "therapy"
        ],
        "DescriptorName":"Neoplasms"
      },
      {
        "QualifierName":[
          "complications",
          "diagnosis"
        ],
     

In [60]:
all_papers = []
for paper in papers:
    d = {}
    d['PMID'] = int(paper['MedlineCitation']['PMID'])
    d['Title'] = paper['MedlineCitation']['Article']['ArticleTitle']
    d['Date'] = pd.to_datetime([paper['MedlineCitation']['Article']['ArticleDate'][0]['Month'],
                                paper['MedlineCitation']['Article']['ArticleDate'][0]['Day'],
                                paper['MedlineCitation']['Article']['ArticleDate'][0]['Year']])
    try:
        d['Abstract'] = paper['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
    except:
        d['Abstract'] = None
    
    all_papers.append(d)

pmdf = pd.DataFrame(all_papers)

In [61]:
pmdf

,Abstract,Date,PMID,Title
0,Fever is defined as a body temperature above 3...,"[11, 27, 2012]",23199412,[Fever and cancer: components of diagnosis for...
1,"To review the incidence of UTIs, post-operativ...","[02, 25, 2012]",22367718,Urinary tract infections and post-operative fe...
2,Considering that the incidence of fever may re...,"[11, 28, 2012]",23199670,Fever effects and treatment in critical care: ...
3,"The definition of fever, and thus fever and ne...","[11, 28, 2012]",23193083,Different fever definitions and the rate of fe...
4,The purpose of the study was to assess whether...,"[03, 14, 2012]",23412282,Is immediate postoperative fever related to dr...
5,While fever is known to occur in invertebrates...,"[11, 18, 2012]",23160839,A review of the physiology of fever in birds.
6,None,"[11, 14, 2012]",23150031,An adolescent boy with acute kidney injury and...
7,Fever happens frequently in colon cancer but i...,"[10, 03, 2012]",23040819,[Fever as the presenting manifestation of colo...
8,Fever is typically considered part of the infl...,"[09, 28, 2012]",23024066,"Effect of fever on hospital presentation, diag..."
9,A young infant with fever and oral ulcerations...,"[09, 26, 2012]",23013324,"Fever, irritability, and ulcerations in an inf..."
